In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('C:/Users/Iasonas/PycharmProjects/EntityMatching_Thesis/DataFusion/tutorials')

import fuse
fuse_env = fuse.Fuse(verbose=True)
fuse_session = fuse_env.get_session('test')

MATCHER='DEDUPE'
# MATCHER='AHC'

C:\Python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\Iasonas\AppData\Roaming\Python\Python38\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
INFO:gensim.models.utils_any2vec:loading projection weights from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:gensim.models.utils_any2vec:loaded (3000000, 300) matrix from C:/Users/Iasonas/Downloads/GoogleNews-vectors-negative300.bin
INFO:global:Connection established to database.


In [2]:
if MATCHER=='DEDUPE':
    from matching.dedupe import DedupeMatcher
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                                  'fusion_test', matching_strategy=DedupeMatcher)

if MATCHER=='AHC':
    from matching.ahc_matcher import AgglomerativeHierarchicalClustering
    dataset = fuse_session.ingest('../../datasets/current/observed.json',
                               'fusion_test', matching_strategy=AgglomerativeHierarchicalClustering)


INFO:dedupe.api:Predicate set:
INFO:dedupe.api:(TfidfTextCanopyPredicate: (0.6, year), LevenshteinCanopyPredicate: (3, university), LevenshteinCanopyPredicate: (4, university))
INFO:dedupe.api:(TfidfNGramCanopyPredicate: (0.2, university), SimplePredicate: (sameFiveCharStartPredicate, degree))
INFO:dedupe.api:(SimplePredicate: (commonFourGram, year), SimplePredicate: (sameThreeCharStartPredicate, degree), LevenshteinCanopyPredicate: (4, university))
INFO:dedupe.api:(SimplePredicate: (nearIntegersPredicate, year), SimplePredicate: (suffixArray, university))
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:Predicate set:
INFO:dedupe.api:(LevenshteinCanopyPredicate: (2, company), TfidfNGramCanopyPredicate: (0.4, year), SimplePredicate: (commonSixGram, company))
INFO:dedupe.api:(SimplePredicate: (firstIntegerPredicate, year), TfidfTextCanopyPredicate: (0.6, title))


In [3]:
fuse_session.train_matchers('../../datasets/current/matched.json')

In [4]:
# x=dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['university'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Bsc','undergraduate_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','doctoral_degree')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('doctorate','Bsc')
# dataset.entity_attribute_matchers['Education'].attribute_to_simrank_matcher['degree'].get_simrank_score_of_tokens('Msc','Bsc')
# x=dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].cluster_simrank_buffer
# {k: round(v, 3) for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('manager','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('coder','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','developer')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('analyst','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('doctor','manager')
# dataset.entity_attribute_matchers['Working Experience'].attribute_to_simrank_matcher['title'].get_simrank_score_of_tokens('clinician','doctor')

In [5]:
fuse_session.match_observations()

In [6]:
tr_clusters = fuse_session.find_true_clusters()


INFO:global:Running truth discovery for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Education': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Location': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Name': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Working Experience': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Education': {'pipl.com': 0.9123614537342654, 'find.jobs': 1.1440746627448668, 'internsteps.com': 1.1440746627448668, 'pro-profiles.com': 0.9400072584914709}, 'Location': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589}, 'Name': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'pro-profil

In [7]:
true_facts = fuse_session.find_true_facts(persist=True, no_weights=False)

INFO:global:Running fusion for 10 iterations.
INFO:global:Iteration = 0
INFO:global:Source weights = {'Education': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Location': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Name': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}, 'Working Experience': {'pipl.com': 1.0, 'find.jobs': 1.0, 'internsteps.com': 1.0, 'pro-profiles.com': 1.0}}
INFO:global:Iteration = 1
INFO:global:Source weights = {'Education': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589}, 'Location': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'pro-profiles.com': 4.595119850134589}, 'Name': {'pipl.com': 4.595119850134589, 'find.jobs': 4.595119850134589, 'internsteps.com': 4.595119850134589, 'pro-profiles.com': 4.59

In [8]:
true_facts_sort = sorted(true_facts, key=lambda tf: (tf.ent_id, tf.ent_attr), reverse=False)
for f in true_facts_sort:
    print(f.ent_id, f.ent_attr, f.canonicalTuple)

# for f in true_facts:
#     print(f.canonicalTuple)

0 Education {'year': '2010', 'degree': 'Msc', 'university': 'UC_Berkley'}
0 Location {'location': 'Langtown'}
0 Name {'name': 'Weslfy Wilsonn'}
0 Working Experience {'company': 'Lopez-Smith', 'title': 'developer', 'years': '1995-1996'}
1 Education {'year': '1997', 'degree': 'Msc', 'university': 'UMass'}
1 Location {'location': 'BENNETTFURT'}
1 Name {'name': 'Adam O.'}
1 Working Experience {'company': 'Jones-Mckenzie', 'title': 'analyst', 'years': '1992-1995'}
2 Education {'year': '2014', 'degree': 'Phd', 'university': 'UMass'}
2 Location {'location': 'HORNFORT'}
2 Name {'name': 'Sharo Martinez'}
2 Working Experience {'company': 'Richards LLC', 'title': 'doctor', 'years': '2017-2020'}
3 Education {'year': '1992', 'degree': 'Phd', 'university': 'UC_Berkley'}
3 Location {'location': 'BAILEYFURT'}
3 Name {'name': 'Richard Wright'}
3 Working Experience {'company': 'Forbes and Sons', 'title': 'doctor', 'years': '1991-1995'}
4 Education {'year': '2003', 'degree': 'Msc', 'university': 'UCSC'}


In [9]:
eval = fuse_session.evaluate(inferred_facts=true_facts, grd_path='../../datasets/current/eval.json')
print(eval)

Matched = 201.00, Unmatched true = 30.00, Unmatched inferred = 0.00, Total = 231.00
Unmatched true facts:
[(0, 'Education', {'degree': 'Msc', 'university': 'UMass', 'year': '2019'}),
 (2,
  'Working Experience',
  {'company': 'Garcia, Ross and Leonard',
   'title': 'doctor',
   'years': '2017-2020'}),
 (8, 'Education', {'degree': 'Bsc', 'university': 'UMass', 'year': '1997'}),
 (10, 'Education', {'degree': 'Phd', 'university': 'UMass', 'year': '1995'}),
 (15, 'Education', {'degree': 'Msc', 'university': 'UMass', 'year': '2001'}),
 (15,
  'Working Experience',
  {'company': 'Mejia, Morgan and Olsen',
   'title': 'doctor',
   'years': '1993-2015'}),
 (16, 'Education', {'degree': 'Bsc', 'university': 'UMass', 'year': '2016'}),
 (17,
  'Education',
  {'degree': 'Phd', 'university': 'UC_Berkley', 'year': '2005'}),
 (17,
  'Working Experience',
  {'company': 'Hancock Ltd', 'title': 'analyst', 'years': '1994-2015'}),
 (17,
  'Working Experience',
  {'company': 'Porter-Todd', 'title': 'doctor'